# Градиентный бустинг своими руками

**Внимание:** в тексте задания произошли изменения - поменялось число деревьев (теперь 50), правило изменения величины шага в задании 3 и добавился параметр `random_state` у решающего дерева. Правильные ответы не поменялись, но теперь их проще получить. Также исправлена опечатка в функции `gbm_predict`.

В этом задании будет использоваться датасет `boston` из `sklearn.datasets`. Оставьте последние 25% объектов для контроля качества, разделив `X` и `y` на `X_train`, `y_train` и `X_test`, `y_test`.

Целью задания будет реализовать простой вариант градиентного бустинга над регрессионными деревьями для случая квадратичной функции потерь.

In [1]:
import sklearn.datasets
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
boston = sklearn.datasets.load_boston()
boston.keys()
X = boston.data
y = boston.target

In [2]:
X_train = boston.data[:int(boston.data.shape[0]*0.75),::]
y_train = boston.target[:int(boston.data.shape[0]*0.75)]

X_test = boston.data[int(boston.data.shape[0]*0.75):,::]
y_test = boston.target[int(boston.data.shape[0]*0.75):]

## Задание 1

Как вы уже знаете из лекций, **бустинг** - это метод построения композиций базовых алгоритмов с помощью последовательного добавления к текущей композиции нового алгоритма с некоторым коэффициентом. 

Градиентный бустинг обучает каждый новый алгоритм так, чтобы он приближал антиградиент ошибки по ответам композиции на обучающей выборке. Аналогично минимизации функций методом градиентного спуска, в градиентном бустинге мы подправляем композицию, изменяя алгоритм в направлении антиградиента ошибки.

Воспользуйтесь формулой из лекций, задающей ответы на обучающей выборке, на которые нужно обучать новый алгоритм (фактически это лишь чуть более подробно расписанный градиент от ошибки), и получите частный ее случай, если функция потерь `L` - квадрат отклонения ответа композиции `a(x)` от правильного ответа `y` на данном `x`.

Если вы давно не считали производную самостоятельно, вам поможет таблица производных элементарных функций (которую несложно найти в интернете) и правило дифференцирования сложной функции. После дифференцирования квадрата у вас возникнет множитель 2 — т.к. нам все равно предстоит выбирать коэффициент, с которым будет добавлен новый базовый алгоритм, проигноируйте этот множитель при дальнейшем построении алгоритма.

$$ a(x) - y$$

## Задание 2

Заведите массив для объектов `DecisionTreeRegressor` (будем их использовать в качестве базовых алгоритмов) и для вещественных чисел (это будут коэффициенты перед базовыми алгоритмами). 

В цикле от обучите последовательно 50 решающих деревьев с параметрами `max_depth=5` и `random_state=42` (остальные параметры - по умолчанию). В бустинге зачастую используются сотни и тысячи деревьев, но мы ограничимся 50, чтобы алгоритм работал быстрее, и его было проще отлаживать (т.к. цель задания разобраться, как работает метод). Каждое дерево должно обучаться на одном и том же множестве объектов, но ответы, которые учится прогнозировать дерево, будут меняться в соответствие с полученным в задании 1 правилом. 

Попробуйте для начала всегда брать коэффициент равным 0.9. Обычно оправдано выбирать коэффициент значительно меньшим - порядка 0.05 или 0.1, но т.к. в нашем учебном примере на стандартном датасете будет всего 50 деревьев, возьмем для начала шаг побольше.

В процессе реализации обучения вам потребуется функция, которая будет вычислять прогноз построенной на данный момент композиции деревьев на выборке `X`:

```
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]
(считаем, что base_algorithms_list - список с базовыми алгоритмами, coefficients_list - список с коэффициентами перед алгоритмами)
```

Эта же функция поможет вам получить прогноз на контрольной выборке и оценить качество работы вашего алгоритма с помощью `mean_squared_error` в `sklearn.metrics`. 

Возведите результат в степень 0.5, чтобы получить `RMSE`. Полученное значение `RMSE` — **ответ в пункте 2**.

In [3]:
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]

In [4]:
def write_answer(answer, file):
    with open(file, "w") as fout:
        fout.write(str(answer))

In [5]:
base_algorithms_list = []
coefficients_list = [0.9 for i in range(X_train.shape[0])] 
base_algorithms_list.append(sklearn.tree.DecisionTreeRegressor(max_depth=5, random_state=42))
base_algorithms_list[0].fit(X_train,y_train)
gradient = base_algorithms_list[0].predict(X_train)[0] - y_train
gbm_predict(X_train)
for i in np.arange(1,50):
        base_algorithms_list.append(sklearn.tree.DecisionTreeRegressor(max_depth=5, random_state=42))
        base_algorithms_list[i].fit(X_train,gradient)
        gradient = -(gbm_predict(X_train) - y_train)
        print(mean_squared_error(y_train, gbm_predict(X_train)))
rmse = mean_squared_error (y_test, gbm_predict(X_test)) ** 0.5

78.21581613720319
6.053802668603867
3.0788583579119915
2.1234277189229642
1.4043729237091789
1.0449552208171655
0.6718226461260148
0.49952057508906555
0.35218204682216664
0.2889922320622579
0.1984452548657375
0.1459751972943546
0.11045479418616698
0.08873097417775294
0.07272042240399272
0.0561658685739755
0.04265208395104802
0.028528788301136702
0.020589484057948138
0.016971165489826693
0.011030487621622113
0.008433798633814767
0.006674893231537918
0.006084375708883288
0.004474324685445996
0.003576672903013986
0.0024877488516549503
0.0019207464152065546
0.0016256826608843917
0.0015615969054257734
0.0011817942345275685
0.0008060965486453538
0.0006625072974150068
0.0005805707807542929
0.00048455518596395296
0.0003591349171038411
0.00024232901008304424
0.00021044857338677845
0.00015017624787862018
0.0001104293844458879
9.540331504392428e-05
6.573113001742692e-05
5.164589963410974e-05
3.664985853628844e-05
3.071078659555323e-05
2.1227825927230706e-05
1.3744315699222556e-05
1.15414756273102

In [6]:
write_answer(rmse, '1.txt')

## Задание 3

Вас может также беспокоить, что двигаясь с постоянным шагом, вблизи минимума ошибки ответы на обучающей выборке меняются слишком резко, перескакивая через минимум. 

Попробуйте уменьшать вес перед каждым алгоритмом с каждой следующей итерацией по формуле `0.9 / (1.0 + i)`, где `i` - номер итерации (от 0 до 49). Используйте качество работы алгоритма как **ответ в пункте 3**. 

В реальности часто применяется следующая стратегия выбора шага: как только выбран алгоритм, подберем коэффициент перед ним численным методом оптимизации таким образом, чтобы отклонение от правильных ответов было минимальным. Мы не будем предлагать вам реализовать это для выполнения задания, но рекомендуем попробовать разобраться с такой стратегией и реализовать ее при случае для себя.

In [7]:
base_algorithms_list = []
coefficients_list = [0.9/(1+i) for i in range(X_train.shape[0])] 
base_algorithms_list.append(sklearn.tree.DecisionTreeRegressor(max_depth=5, random_state=42))
base_algorithms_list[0].fit(X_train,y_train)
gradient = base_algorithms_list[0].predict(X_train)[0] - y_train
gbm_predict(X_train)
for i in np.arange(1,50):
        base_algorithms_list.append(sklearn.tree.DecisionTreeRegressor(max_depth=5, random_state=42))
        base_algorithms_list[i].fit(X_train,gradient)
        gradient = -(gbm_predict(X_train) - y_train)
        print(mean_squared_error(y_train, gbm_predict(X_train)))
rmse = mean_squared_error (y_test, gbm_predict(X_test)) ** 0.5

30.115695125417776
16.664173830544275
11.304027837377012
8.593398144790948
6.9484711702159645
5.858510900387708
5.09879603231323
4.541756413697509
4.060455793322275
3.7243608466604785
3.461791876732583
3.214496192832206
3.0108046391163734
2.821243634365267
2.6578101825892295
2.550112172587875
2.4291091473028965
2.334530650866501
2.25264841292607
2.163666153328301
2.0930795486755844
2.0288223163810724
1.962027326443072
1.9123659447907646
1.871543307384763
1.831543568785137
1.785240154973553
1.733177647555019
1.6938529787827286
1.659156312087655
1.617610601126241
1.5838831761971548
1.5528762228937294
1.527898617659955
1.5008432333841653
1.4746477311950812
1.4509283392688757
1.4274117012306877
1.405465844232421
1.3844367492462548
1.3687086150788463
1.3492170283632996
1.3309507884158203
1.3132389335751908
1.2969370673476681
1.2815750366773662
1.2676416727547475
1.2504064218233872
1.2379685974122068


In [8]:
write_answer(rmse, '2.txt')

## Задание 4

Реализованный вами метод - градиентный бустинг над деревьями - очень популярен в машинном обучении. Он представлен как в самой библиотеке `sklearn`, так и в сторонней библиотеке `XGBoost`, которая имеет свой питоновский интерфейс. На практике `XGBoost` работает заметно лучше `GradientBoostingRegressor` из `sklearn`, но для этого задания вы можете использовать любую реализацию. 

Исследуйте, переобучается ли градиентный бустинг с ростом числа итераций (и подумайте, почему), а также с ростом глубины деревьев. На основе наблюдений выпишите через пробел номера правильных из приведенных ниже утверждений в порядке возрастания номера (это будет **ответ в п.4**):

    1. С увеличением числа деревьев, начиная с некоторого момента, качество работы градиентного бустинга не меняется существенно.

    2. С увеличением числа деревьев, начиная с некоторого момента, градиентный бустинг начинает переобучаться.

    3. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга на тестовой выборке начинает ухудшаться.

    4. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга перестает существенно изменяться

## Задание 5

Сравните получаемое с помощью градиентного бустинга качество с качеством работы линейной регрессии. 

Для этого обучите `LinearRegression` из `sklearn.linear_model` (с параметрами по умолчанию) на обучающей выборке и оцените для прогнозов полученного алгоритма на тестовой выборке `RMSE`. Полученное качество - ответ в **пункте 5**. 

В данном примере качество работы простой модели должно было оказаться хуже, но не стоит забывать, что так бывает не всегда. В заданиях к этому курсу вы еще встретите пример обратной ситуации.

In [9]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train,y_train)
rmse = mean_squared_error (y_test, lr.predict(X_test)) ** 0.5
write_answer(rmse, '4.txt')

In [10]:
print (rmse)

8.270468034938213
